In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException, ElementClickInterceptedException, TimeoutException
)
from webdriver_manager.chrome import ChromeDriverManager
import requests
import time
import random
import pandas as pd
import os
from urllib.parse import urljoin
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Alignment

# --------------------------
# 辅助函数1：清理文件名（避免特殊字符）
# --------------------------
def clean_filename(filename):
    invalid_chars = ['\\', '/', ':', '*', '?', '"', '<', '>', '|']
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    return filename[:50]

# --------------------------
# 辅助函数2：下载商品图片到本地
# --------------------------
def download_product_image(img_url, save_dir, product_name, page_num):
    if not img_url or img_url.strip() == "":
        return "无图片URL"
    
    base_url = "https://www.dearpet.tw"
    full_img_url = urljoin(base_url, img_url)
    clean_name = clean_filename(product_name)
    img_filename = f"第{page_num}页_{clean_name}.jpg"
    save_path = os.path.join(save_dir, img_filename)

    if os.path.exists(save_path):
        return save_path
    
    time.sleep(random.uniform(0.5, 1.5))
    try:
        headers = {
            "User-Agent": f"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 120)}.0.0.0 Safari/537.36"
        }
        response = requests.get(full_img_url, headers=headers, timeout=10, stream=True)
        response.raise_for_status()
        
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        return save_path
    
    except Exception as e:
        return f"下载失败：{str(e)[:20]}"

# --------------------------
# 辅助函数3：生成带嵌入图片的Excel
# --------------------------
def generate_excel_with_images(products, img_save_dir):
    wb = Workbook()
    ws = wb.active
    ws.title = "宠物商品数据"

    # 表头设置
    headers = ["页面", "商品名称", "价格", "商品图片URL", "商品图片"]
    for col, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col, value=header)
        cell.alignment = Alignment(horizontal="center", vertical="center")

    # 填充数据+插入图片
    for row, product in enumerate(products, 2):
        ws.cell(row=row, column=1, value=product["页面"])
        ws.cell(row=row, column=2, value=product["商品名称"])
        ws.cell(row=row, column=3, value=product["价格"])
        ws.cell(row=row, column=4, value=product["商品图片URL"])

        # 插入图片
        local_img_path = product["本地图片路径"]
        if local_img_path not in ["无图片", "无图片URL"] and "下载失败" not in local_img_path and os.path.exists(local_img_path):
            img = Image(local_img_path)
            img.width = 80
            img.height = 80
            ws.add_image(img, anchor=f"E{row}")
            ws.row_dimensions[row].height = 60  # 适配图片高度
        else:
            ws.cell(row=row, column=5, value=local_img_path)

    # 调整列宽
    ws.column_dimensions["A"].width = 8
    ws.column_dimensions["B"].width = 35
    ws.column_dimensions["C"].width = 15
    ws.column_dimensions["D"].width = 45
    ws.column_dimensions["E"].width = 12

    # 保存Excel
    excel_path = "dearpet_products_with_images.xlsx"
    wb.save(excel_path)
    print(f"\n📊 带图片的Excel已保存到：{excel_path}")
    return excel_path

# --------------------------
# 主爬取函数（翻页使用CSS选择器）
# --------------------------
def crawl_dearpet_full(start_url, max_pages=None):
    # 浏览器初始化（强化反爬伪装）
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    chrome_options.add_argument("window-size=1280,720")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 隐藏webdriver特征
    chrome_options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(90, 120)}.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(
        service=ChromeService(ChromeDriverManager().install()),
        options=chrome_options
    )

    # 图片文件夹
    img_save_dir = "dearpet_product_images"
    if not os.path.exists(img_save_dir):
        os.makedirs(img_save_dir)

    all_products = []
    current_page = 1

    try:
        driver.get(start_url)
        time.sleep(random.uniform(2, 4))  # 首次加载等待

        while True:
            print(f"\n========== 爬取 第 {current_page} 页 ==========")
            # 循环滚动加载所有商品（处理动态加载）
            last_product_count = 0
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(random.uniform(1.5, 2.5))  # 等待新商品加载
                current_products = driver.find_elements(By.CSS_SELECTOR, ".js-quick-cart-item")  # 商品容器CSS
                if len(current_products) == last_product_count:
                    print(f"当前页商品加载完成，共 {len(current_products)} 个")
                    break  # 商品数量不变，说明加载完毕
                last_product_count = len(current_products)
            product_items = current_products

            # 提取当前页商品信息
            for idx, item in enumerate(product_items, 1):
                try:
                    # 1. 商品名称（显式等待元素加载）
                    name_elem = WebDriverWait(item, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".info-box-inner-wrapper .title.text-primary-color"))
                    )
                    name_text = name_elem.text.strip() if name_elem.text else ""
                    product_name = name_text if name_text else f"未知名称_{current_page}_{idx}"

                    # 2. 商品价格
                    price_elem = WebDriverWait(item, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".quick-cart-price .global-primary.dark-primary.price.sl-price"))
                    )
                    price_text = price_elem.text.strip() if price_elem.text else ""
                    product_price = price_text if price_text else "未知价格"

                    # 3. 商品图片URL（适配懒加载）
                    img_elem = WebDriverWait(item, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".boxify-image-wrap img"))
                    )
                    # 优先从srcset提取，其次data-src，最后src
                    srcset = img_elem.get_attribute("srcset") or ""
                    if srcset.strip():
                        product_img_url = srcset.split(",")[0].strip().split(" ")[0]
                    else:
                        product_img_url = img_elem.get_attribute("data-src") or img_elem.get_attribute("src") or "无图片URL"
                    product_img_url = product_img_url.strip() if product_img_url else "无图片URL"

                    # 4. 下载图片
                    local_img_path = download_product_image(
                        product_img_url, img_save_dir, product_name, current_page
                    ) if product_img_url != "无图片URL" else "无图片URL"

                    # 保存数据
                    all_products.append({
                        "页面": current_page,
                        "商品名称": product_name,
                        "价格": product_price,
                        "商品图片URL": product_img_url,
                        "本地图片路径": local_img_path
                    })
                    print(f"✅  已处理第{idx}个商品：{product_name}")

                except TimeoutException:
                    print(f"⚠️  第{idx}个商品元素加载超时，跳过")
                    continue
                except Exception as e:
                    print(f"⚠️  第{idx}个商品处理失败：{str(e)}")
                    continue

            # --------------------------
            # 翻页逻辑（使用CSS选择器）
            # --------------------------
            try:
                # 滚动到分页区域（确保翻页按钮加载）
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(random.uniform(1.5, 2.5))

                # 关键：翻页按钮的CSS选择器（匹配包含右箭头图标的按钮）
                # 说明：ul.pagination 是分页容器，a.Label 是按钮，i.fa-angle-right 是右箭头图标
                next_page_css = "#Content .right-c-box .text-center ul li a:has(i.fa-angle-right)"
                # 等待按钮可点击
                next_page_btn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, next_page_css))
                )

                # 判断是否为最后一页（通过按钮是否可点击+当前URL）
                current_url = driver.current_url
                if f"page={current_page}" in current_url and not driver.find_elements(By.CSS_SELECTOR, next_page_css):
                    print("\n已到最后一页，无下一页按钮")
                    break

                # 点击下一页（用JS点击避免被遮挡）
                driver.execute_script("arguments[0].click();", next_page_btn)
                time.sleep(random.uniform(2, 3))  # 等待页面加载
                current_page += 1

                # 控制最大页数
                if max_pages and current_page > max_pages:
                    print(f"\n已达最大爬取页数 {max_pages}，停止")
                    break

            except (NoSuchElementException, TimeoutException):
                print("\n未找到有效的下一页按钮，停止爬取")
                break
            except ElementClickInterceptedException:
                print("\n翻页按钮被遮挡，尝试刷新页面...")
                driver.refresh()
                time.sleep(random.uniform(2, 3))
                continue
            except Exception as e:
                print(f"\n翻页失败：{str(e)}，停止爬取")
                break

    except Exception as e:
        print(f"\n❌ 整体爬取异常：{str(e)}")
    finally:
        # 生成结果文件
        if all_products:
            df = pd.DataFrame(all_products).drop_duplicates(subset=["商品名称"], keep="first")
            csv_path = "dearpet_products_backup.csv"
            df.to_csv(csv_path, index=False, encoding="utf-8-sig")
            generate_excel_with_images(df.to_dict("records"), img_save_dir)
            print(f"\n🎉 爬取完成！共获取 {len(df)} 个有效商品")
            print(f"💾 纯文本备份：{csv_path}")
        else:
            print("\n⚠️  未爬取到有效商品数据")

        driver.quit()
        print(f"\n🖥️  浏览器已关闭，图片路径：{os.path.abspath(img_save_dir)}")

# --------------------------
# 执行爬取
# --------------------------
if __name__ == "__main__":
    start_url = "https://www.dearpet.tw/products?page=1&sort_by=&order_by=&limit=72"  # 起始页
    max_crawl_pages = None  # 爬取全部页（如需限制，改为数字如3）

    print(f"🚀 开始爬取：{start_url}")
    print(f"📄 爬取页数：{'全部' if not max_crawl_pages else max_crawl_pages}")
    crawl_dearpet_full(start_url, max_crawl_pages)

🚀 开始爬取：https://www.dearpet.tw/products?page=1&sort_by=&order_by=&limit=72
📄 爬取页数：全部

========== 爬取 第 1 页 ==========
当前页商品加载完成，共 72 个
✅  已处理第1个商品：客製寵物足跡回憶相框
✅  已处理第2个商品：寵物祈念台 回憶之橋(照片款)
✅  已处理第3个商品：毛孩骨灰時光沙漏祈念台組合
✅  已处理第4个商品：毛孩骨灰時光沙漏
✅  已处理第5个商品：大倉陶園寵物骨灰罐
✅  已处理第6个商品：寵物祈念銅磬Echo
✅  已处理第7个商品：日本美濃燒防潮寵物骨灰罐(4寸)(30公斤以下)
✅  已处理第8个商品：和風防潮寵物骨灰罐(4寸)(30公斤以下)
✅  已处理第9个商品：日本美濃燒防潮寵物骨灰罐(5寸)(30公斤以上)
✅  已处理第10个商品：拱形水晶玻璃寵物祈念牌位
✅  已处理第11个商品：守護天使祈念銅磬
✅  已处理第12个商品：日本美濃燒防潮寵物骨灰罐(3寸)(10公斤以下)
✅  已处理第13个商品：祈念蓮花擺飾
✅  已处理第14个商品：寵物毛髮紀念皮革鑰匙圈 照片款
✅  已处理第15个商品：杯型寵物祈念提燈
✅  已处理第16个商品：祈念線香 櫻人
✅  已处理第17个商品：和風寵物祈念花瓶
✅  已处理第18个商品：小肉球浮雕款 祈念花瓶
✅  已处理第19个商品：親愛毛寶貝時光膠囊鑰匙圈刻印款(附收納袋)
✅  已处理第20个商品：祈念豪華組合
✅  已处理第21个商品：防潮寵物骨灰罐 極光(小型)(5公斤以下)
✅  已处理第22个商品：寵物毛髮紀念皮革鑰匙圈
⚠️  第23个商品元素加载超时，跳过
⚠️  第24个商品元素加载超时，跳过
⚠️  第25个商品元素加载超时，跳过
⚠️  第26个商品元素加载超时，跳过
⚠️  第27个商品元素加载超时，跳过
⚠️  第28个商品元素加载超时，跳过
✅  已处理第29个商品：特殊商品專用下單頁面
✅  已处理第30个商品：彩虹訂製寵物骨灰罐(5寸)
✅  已处理第31个商品：彩虹訂製寵物骨灰罐(4寸)
✅  已处理第32个商品：彩虹訂製寵物骨灰罐(3.5寸)
✅  已处理第33个商品：彩虹訂製寵物骨灰罐(3寸)
✅  已处理第34个商品：彩虹訂製寵物骨灰罐